# GroupKFold Validation Experiment

**Hypothesis**: Our LOO validation is TOO OPTIMISTIC. It gives us CV scores that don't translate to LB.

**Key Insight from Kaggle Kernels**: The "mixall" kernel (8 votes, claims "good CV/LB") uses GroupKFold (5 splits) instead of LOO!

**Approach**:
- Override `generate_leave_one_out_splits` and `generate_leave_one_ramp_out_splits` to use GroupKFold (5 splits)
- Use the SAME model as exp_026 (MLP+LGBM ensemble with weighted loss)
- Compare the new CV score with the old LOO CV score

**Expected Result**: GroupKFold CV will be HIGHER (worse) than LOO CV, but will better correlate with LB.

**Baseline**: exp_026 LOO CV = 0.008465, LB = 0.0887

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
from sklearn.model_selection import GroupKFold
import tqdm
import warnings
import sys
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions with GroupKFold
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

# ORIGINAL LOO FUNCTIONS (for reference)
def generate_leave_one_out_splits_LOO(X, Y):
    """Original LOO - leaves out one solvent at a time"""
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits_LOO(X, Y):
    """Original LORO - leaves out one ramp at a time"""
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

# NEW GROUPKFOLD FUNCTIONS
def generate_leave_one_out_splits(X, Y):
    """GroupKFold (5 splits) on SOLVENT NAME"""
    groups = X["SOLVENT NAME"]
    n_splits = min(5, len(groups.unique()))
    gkf = GroupKFold(n_splits=n_splits)
    for train_idx, test_idx in gkf.split(X, Y, groups):
        yield (X.iloc[train_idx], Y.iloc[train_idx]), (X.iloc[test_idx], Y.iloc[test_idx])

def generate_leave_one_ramp_out_splits(X, Y):
    """GroupKFold (5 splits) on SOLVENT A NAME + SOLVENT B NAME"""
    groups = X["SOLVENT A NAME"].astype(str) + "_" + X["SOLVENT B NAME"].astype(str)
    n_splits = min(5, len(groups.unique()))
    gkf = GroupKFold(n_splits=n_splits)
    for train_idx, test_idx in gkf.split(X, Y, groups):
        yield (X.iloc[train_idx], Y.iloc[train_idx]), (X.iloc[test_idx], Y.iloc[test_idx])

print('Data loading functions defined with GroupKFold')

# Check how many groups we have
X_single, Y_single = load_data("single_solvent")
X_full, Y_full = load_data("full")
print(f'Single solvent: {len(X_single["SOLVENT NAME"].unique())} unique solvents')
print(f'Full data: {len((X_full["SOLVENT A NAME"].astype(str) + "_" + X_full["SOLVENT B NAME"].astype(str)).unique())} unique ramps')

Data loading functions defined with GroupKFold
Single solvent: 24 unique solvents
Full data: 13 unique ramps


In [3]:
# Load feature lookups INCLUDING ACS PCA
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')
print(f'Total features: 5 (kinetic) + {SPANGE_DF.shape[1]} (Spange) + {DRFP_FILTERED.shape[1]} (DRFP) + {ACS_PCA_DF.shape[1]} (ACS PCA) = {5 + SPANGE_DF.shape[1] + DRFP_FILTERED.shape[1] + ACS_PCA_DF.shape[1]}')

Spange: (26, 13), DRFP filtered: (24, 122), ACS PCA: (24, 5)
Total features: 5 (kinetic) + 13 (Spange) + 122 (DRFP) + 5 (ACS PCA) = 145


In [4]:
# Combined Featurizer with Arrhenius kinetics AND ACS PCA
class ACSPCAFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.acs_pca_df = ACS_PCA_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1] + self.acs_pca_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            A_acs = self.acs_pca_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_pca_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
                X_acs = B_acs * (1 - (1-pct)) + A_acs * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
                X_acs = A_acs * (1 - pct) + B_acs * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_pca_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])
    
    def featurize_torch(self, X, flip=False):
        return torch.tensor(self.featurize(X, flip))

print(f'Feature dimension: {ACSPCAFeaturizer().feats_dim}')

Feature dimension: 145


In [5]:
# Weighted Huber Loss - weights SM (target 2) higher
class WeightedHuberLoss(nn.Module):
    def __init__(self, weights=[1.0, 1.0, 2.0]):  # [P2, P3, SM]
        super().__init__()
        self.weights = torch.tensor(weights, dtype=torch.double)
        self.huber = nn.HuberLoss(reduction='none')
    
    def forward(self, pred, target):
        # HuberLoss per element
        huber_loss = self.huber(pred, target)  # Shape: [batch, 3]
        # Apply weights
        weighted_loss = huber_loss * self.weights.to(pred.device)
        return weighted_loss.mean()

print('WeightedHuberLoss defined: weights = [1.0, 1.0, 2.0] for [P2, P3, SM]')

WeightedHuberLoss defined: weights = [1.0, 1.0, 2.0] for [P2, P3, SM]


In [6]:
# MLP Model with Weighted Loss
class WeightedMLPModel(nn.Module):
    def __init__(self, input_dim, hidden_dims=[32, 16], output_dim=3):
        super().__init__()
        layers = []
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, h_dim),
                nn.BatchNorm1d(h_dim),
                nn.ReLU(),
                nn.Dropout(0.2)
            ])
            prev_dim = h_dim
        layers.append(nn.Linear(prev_dim, output_dim))
        layers.append(nn.Sigmoid())
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

print('WeightedMLPModel defined')

WeightedMLPModel defined


In [11]:
# Weighted Loss Ensemble (MLP + LightGBM)
class WeightedLossEnsemble:
    def __init__(self, data='single'):
        self.data = data
        self.featurizer = ACSPCAFeaturizer(mixed=(data == 'full'))
        self.mlp = None
        self.lgb_models = []
        self.scaler_mean = None
        self.scaler_std = None
        
    def train_model(self, X, Y):
        # Featurize
        X_feat = self.featurizer.featurize(X)
        Y_np = Y.values
        
        # Normalize features
        self.scaler_mean = X_feat.mean(axis=0)
        self.scaler_std = X_feat.std(axis=0) + 1e-8
        X_norm = (X_feat - self.scaler_mean) / self.scaler_std
        
        # Train MLP with weighted loss
        self.mlp = WeightedMLPModel(input_dim=X_norm.shape[1]).to(device)
        optimizer = torch.optim.Adam(self.mlp.parameters(), lr=5e-4, weight_decay=1e-4)
        criterion = WeightedHuberLoss(weights=[1.0, 1.0, 2.0])
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)
        
        X_tensor = torch.tensor(X_norm, dtype=torch.double).to(device)
        Y_tensor = torch.tensor(Y_np, dtype=torch.double).to(device)
        dataset = TensorDataset(X_tensor, Y_tensor)
        # Use drop_last=True to avoid batch size 1 issue with BatchNorm
        loader = DataLoader(dataset, batch_size=32, shuffle=True, drop_last=True)
        
        self.mlp.train()
        for epoch in range(200):
            epoch_loss = 0
            for batch_X, batch_Y in loader:
                optimizer.zero_grad()
                pred = self.mlp(batch_X)
                loss = criterion(pred, batch_Y)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
            scheduler.step(epoch_loss)
        
        # Train LightGBM (per-target)
        self.lgb_models = []
        lgb_params = {
            'objective': 'regression',
            'metric': 'mse',
            'learning_rate': 0.03,
            'max_depth': 6,
            'num_leaves': 31,
            'reg_alpha': 0.1,
            'reg_lambda': 0.1,
            'verbose': -1,
            'n_jobs': -1
        }
        for i in range(3):
            model = lgb.LGBMRegressor(**lgb_params, n_estimators=500)
            model.fit(X_norm, Y_np[:, i])
            self.lgb_models.append(model)
    
    def predict(self, X):
        X_feat = self.featurizer.featurize(X)
        X_norm = (X_feat - self.scaler_mean) / self.scaler_std
        
        # MLP prediction
        self.mlp.eval()
        with torch.no_grad():
            X_tensor = torch.tensor(X_norm, dtype=torch.double).to(device)
            mlp_pred = self.mlp(X_tensor).cpu().numpy()
        
        # LightGBM prediction
        lgb_pred = np.column_stack([model.predict(X_norm) for model in self.lgb_models])
        lgb_pred = np.clip(lgb_pred, 0, 1)
        
        # Ensemble (average)
        ensemble_pred = 0.5 * mlp_pred + 0.5 * lgb_pred
        return torch.tensor(ensemble_pred)

print('WeightedLossEnsemble defined (with drop_last=True)')

WeightedLossEnsemble defined (with drop_last=True)


In [8]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = WeightedLossEnsemble(data='single')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:05,  5.42s/it]

2it [00:10,  4.95s/it]

3it [00:14,  4.81s/it]

4it [00:19,  4.74s/it]

5it [00:23,  4.72s/it]

5it [00:23,  4.80s/it]

In [10]:
# Debug: Check fold sizes for full data
X_full, Y_full = load_data("full")
groups_full = X_full["SOLVENT A NAME"].astype(str) + "_" + X_full["SOLVENT B NAME"].astype(str)
n_splits_full = min(5, len(groups_full.unique()))
gkf_full = GroupKFold(n_splits=n_splits_full)

print(f"Full data: {len(X_full)} samples, {len(groups_full.unique())} unique ramps")
print(f"Using {n_splits_full} folds")
print()
for fold_idx, (train_idx, test_idx) in enumerate(gkf_full.split(X_full, Y_full, groups_full)):
    test_groups = groups_full.iloc[test_idx].unique()
    print(f"Fold {fold_idx}: train={len(train_idx)}, test={len(test_idx)}, test_groups={list(test_groups)}")

Full data: 1227 samples, 13 unique ramps
Using 5 folds

Fold 0: train=993, test=234, test_groups=['Ethanol_THF [Tetrahydrofuran]', 'Dihydrolevoglucosenone (Cyrene)_Ethyl Acetate', 'tert-Butanol [2-Methylpropan-2-ol]_Dimethyl Carbonate', 'Methyl Propionate_Ethyl Lactate']
Fold 1: train=980, test=247, test_groups=['Methanol_Ethylene Glycol [1,2-Ethanediol]', 'Acetonitrile_Acetonitrile.Acetic Acid']
Fold 2: train=992, test=235, test_groups=['2,2,2-Trifluoroethanol_Water.2,2,2-Trifluoroethanol', 'DMA [N,N-Dimethylacetamide]_Decanol']
Fold 3: train=964, test=263, test_groups=['Cyclohexane_IPA [Propan-2-ol]', 'Water.Acetonitrile_Acetonitrile', 'MTBE [tert-Butylmethylether]_Butanone [MEK]']
Fold 4: train=979, test=248, test_groups=['1,1,1,3,3,3-Hexafluoropropan-2-ol_2-Methyltetrahydrofuran [2-MeTHF]', '2-Methyltetrahydrofuran [2-MeTHF]_Diethyl Ether [Ether]']


In [12]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = WeightedLossEnsemble(data='full')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:08,  8.63s/it]

2it [00:17,  8.53s/it]

3it [00:25,  8.59s/it]

4it [00:34,  8.50s/it]

5it [00:42,  8.48s/it]

5it [00:42,  8.51s/it]

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
# Calculate CV score with GroupKFold
# Note: GroupKFold changes the order of predictions, so we need to track actuals differently

# For GroupKFold, we need to track which samples were in which fold
X_single, Y_single = load_data("single_solvent")
X_full, Y_full = load_data("full")

# Reconstruct actuals in the same order as predictions
# For single solvent
actuals_single = []
groups_single = X_single["SOLVENT NAME"]
n_splits_single = min(5, len(groups_single.unique()))
gkf_single = GroupKFold(n_splits=n_splits_single)
for train_idx, test_idx in gkf_single.split(X_single, Y_single, groups_single):
    actuals_single.append(Y_single.iloc[test_idx].values)
actuals_single = np.vstack(actuals_single)

# For full data
actuals_full = []
groups_full = X_full["SOLVENT A NAME"].astype(str) + "_" + X_full["SOLVENT B NAME"].astype(str)
n_splits_full = min(5, len(groups_full.unique()))
gkf_full = GroupKFold(n_splits=n_splits_full)
for train_idx, test_idx in gkf_full.split(X_full, Y_full, groups_full):
    actuals_full.append(Y_full.iloc[test_idx].values)
actuals_full = np.vstack(actuals_full)

# Get predictions
preds_single = submission_single_solvent[['target_1', 'target_2', 'target_3']].values
preds_full = submission_full_data[['target_1', 'target_2', 'target_3']].values

# Calculate MSE
mse_single = np.mean((actuals_single - preds_single) ** 2)
mse_full = np.mean((actuals_full - preds_full) ** 2)
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

# Per-target MSE breakdown
mse_p2 = np.mean((actuals_single[:, 0] - preds_single[:, 0]) ** 2)
mse_p3 = np.mean((actuals_single[:, 1] - preds_single[:, 1]) ** 2)
mse_sm = np.mean((actuals_single[:, 2] - preds_single[:, 2]) ** 2)

print(f'\n=== GroupKFold CV SCORE ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single}, {n_splits_single} folds)')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full}, {n_splits_full} folds)')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\n=== PER-TARGET MSE (Single Solvent) ===')
print(f'Product 2 MSE: {mse_p2:.6f}')
print(f'Product 3 MSE: {mse_p3:.6f}')
print(f'SM MSE: {mse_sm:.6f}')

print(f'\n=== COMPARISON ===')
print(f'exp_026 LOO CV: 0.008465, LB: 0.0887')
print(f'exp_034 GroupKFold CV: {overall_mse:.6f}')

if overall_mse > 0.008465:
    ratio = overall_mse / 0.008465
    print(f'\nGroupKFold CV is {ratio:.2f}x HIGHER (worse) than LOO CV')
    print(f'This is EXPECTED - GroupKFold is more pessimistic')
    print(f'\nIf GroupKFold CV correlates better with LB:')
    print(f'  - LOO CV 0.008465 -> LB 0.0887 (ratio: {0.0887/0.008465:.2f}x)')
    print(f'  - GroupKFold CV {overall_mse:.6f} -> LB ??? (expected ratio: closer to 1x)')
else:
    print(f'\nGroupKFold CV is LOWER than LOO CV - unexpected!')